In [72]:
!pip install shap xgboost

# importing common libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

# importing time libraries
from datetime import timedelta
from datetime import datetime

# libraries for baseline algorithm
# from sklearn import svm
# from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model, tree, ensemble
from sklearn.neighbors import KNeighborsClassifier

# libraries for advanced algorithm
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

# libraries for feature selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

Defaulting to user installation because normal site-packages is not writeable


In [73]:
train_path = r"C:\For Internships & Job Applications\personal project\Ecommerce Order & Supply Chain Dataset\archive\Ecommerce Order Dataset\train"
test_path = r"C:\For Internships & Job Applications\personal project\Ecommerce Order & Supply Chain Dataset\archive\Ecommerce Order Dataset\test"

import read_csv_files as rsv

df_train_dict = rsv.read_csv(train_path)
df_test_dict = rsv.read_csv(test_path)

In [74]:
import warnings
warnings.filterwarnings("ignore")

In [75]:
df_train_dict.keys()

dict_keys(['df_Customers.csv', 'df_OrderItems.csv', 'df_Orders.csv', 'df_Payments.csv', 'df_Products.csv'])

In [76]:
df_train_dict.values()

dict_values([        customer_id  customer_zip_code_prefix       customer_city  \
0      hCT0x9JiGXBQ                     58125     varzea paulista   
1      PxA7fv9spyhx                      3112  armacao dos buzios   
2      g3nXeJkGI0Qw                      4119             jandira   
3      EOEsCQ6QlpIg                     18212          uberlandia   
4      mVz5LO2Vd6cL                     88868            ilhabela   
...             ...                       ...                 ...   
89311  bfJtzkBGksdY                     94060              amparo   
89312  4hjsCpj6FC7g                     99835             itapevi   
89313  DLn56oKHl9bv                      2540           sao paulo   
89314  auSfRYUQUOIq                     12288            paulinia   
89315  rhevVNXYJnEs                     93048            brasilia   

      customer_state  
0                 SP  
1                 RJ  
2                 SP  
3                 MG  
4                 SP  
...              ...

In [77]:
tr_customers = df_train_dict["df_Customers.csv"]
tr_orders = df_train_dict["df_Orders.csv"]
tr_items = df_train_dict["df_OrderItems.csv"]
tr_payments = df_train_dict["df_Payments.csv"]
tr_products = df_train_dict["df_Products.csv"]

train_data = [tr_customers, tr_orders, tr_payments, tr_items, tr_products]

In [78]:
for i in train_data:
    print(i.info())
    print(i.head())
    print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               89316 non-null  object
 1   customer_zip_code_prefix  89316 non-null  int64 
 2   customer_city             89316 non-null  object
 3   customer_state            89316 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.7+ MB
None
    customer_id  customer_zip_code_prefix       customer_city customer_state
0  hCT0x9JiGXBQ                     58125     varzea paulista             SP
1  PxA7fv9spyhx                      3112  armacao dos buzios             RJ
2  g3nXeJkGI0Qw                      4119             jandira             SP
3  EOEsCQ6QlpIg                     18212          uberlandia             MG
4  mVz5LO2Vd6cL                     88868            ilhabela             SP


<class 'pandas.core.frame.DataFrame'>
RangeIndex:

In [79]:
test_customers = df_test_dict["df_Customers.csv"]
test_orders = df_test_dict["df_Orders.csv"]
test_items = df_test_dict["df_OrderItems.csv"]
test_payments = df_test_dict["df_Payments.csv"]
test_products = df_test_dict["df_Products.csv"]

test_data = [test_customers, test_orders, test_items, test_payments, test_products]

In [80]:
for i in test_data:
    print(i.info())
    print(i.head())
    print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38279 entries, 0 to 38278
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               38279 non-null  object
 1   customer_zip_code_prefix  38279 non-null  int64 
 2   customer_city             38279 non-null  object
 3   customer_state            38279 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB
None
    customer_id  customer_zip_code_prefix        customer_city customer_state
0  I74lXDOfoqsp                      6020              goiania             GO
1  47TuLHF2s7X5                     23020               viamao             RS
2  dQ0dqI8Qwlj8                     75094             campinas             SP
3  iQCmWhNkIczb                     89284  santana de parnaiba             SP
4  Dp2g6JH8tO5Z                     39810             aripuana             MT


<class 'pandas.core.frame.DataFrame'>
Range

In [81]:
## EDA
## Merging training dataframes

tr_products = tr_products.drop_duplicates()

In [82]:
tr_data = tr_orders.merge(tr_customers, on="customer_id", how="left")
tr_data = tr_data.merge(tr_items, on="order_id", how="left")
tr_data = tr_data.merge(tr_payments, on="order_id", how="left")
tr_data = tr_data.merge(tr_products, on="product_id", how="left")
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       89316 non-null  object 
 1   customer_id                    89316 non-null  object 
 2   order_status                   89316 non-null  object 
 3   order_purchase_timestamp       89316 non-null  object 
 4   order_approved_at              89307 non-null  object 
 5   order_delivered_timestamp      87427 non-null  object 
 6   order_estimated_delivery_date  89316 non-null  object 
 7   customer_zip_code_prefix       89316 non-null  int64  
 8   customer_city                  89316 non-null  object 
 9   customer_state                 89316 non-null  object 
 10  product_id                     89316 non-null  object 
 11  seller_id                      89316 non-null  object 
 12  price                          89316 non-null 

In [83]:
## Merging test dataframes
ts_data = test_orders.merge(test_customers, on="customer_id", how="left")
ts_data = ts_data.merge(test_items, on="order_id", how="left")
ts_data = ts_data.merge(test_payments, on="order_id", how="left")
test_products = test_products.drop_duplicates()
ts_data = ts_data.merge(test_products, on="product_id", how="left")
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38279 entries, 0 to 38278
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   order_id                  38279 non-null  object 
 1   customer_id               38279 non-null  object 
 2   order_purchase_timestamp  38279 non-null  object 
 3   order_approved_at         38272 non-null  object 
 4   customer_zip_code_prefix  38279 non-null  int64  
 5   customer_city             38279 non-null  object 
 6   customer_state            38279 non-null  object 
 7   product_id                38279 non-null  object 
 8   seller_id                 38279 non-null  object 
 9   price                     38279 non-null  float64
 10  shipping_charges          38279 non-null  float64
 11  payment_sequential        38279 non-null  int64  
 12  payment_type              38279 non-null  object 
 13  payment_installments      38279 non-null  int64  
 14  paymen

In [84]:
print("Null values in train data: ")
print(tr_data.isna().sum())
print("\n")
print("Null values in test data: ")
print(ts_data.isna().sum())

Null values in train data: 
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   9
order_delivered_timestamp        1889
order_estimated_delivery_date       0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
product_id                          0
seller_id                           0
price                               0
shipping_charges                    0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
product_category_name             308
product_weight_g                   15
product_length_cm                  15
product_height_cm                  15
product_width_cm                   15
dtype: int64


Null values in test data: 
order_id                      0
customer_id             

In [85]:
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       89316 non-null  object 
 1   customer_id                    89316 non-null  object 
 2   order_status                   89316 non-null  object 
 3   order_purchase_timestamp       89316 non-null  object 
 4   order_approved_at              89307 non-null  object 
 5   order_delivered_timestamp      87427 non-null  object 
 6   order_estimated_delivery_date  89316 non-null  object 
 7   customer_zip_code_prefix       89316 non-null  int64  
 8   customer_city                  89316 non-null  object 
 9   customer_state                 89316 non-null  object 
 10  product_id                     89316 non-null  object 
 11  seller_id                      89316 non-null  object 
 12  price                          89316 non-null 

In [86]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38279 entries, 0 to 38278
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   order_id                  38279 non-null  object 
 1   customer_id               38279 non-null  object 
 2   order_purchase_timestamp  38279 non-null  object 
 3   order_approved_at         38272 non-null  object 
 4   customer_zip_code_prefix  38279 non-null  int64  
 5   customer_city             38279 non-null  object 
 6   customer_state            38279 non-null  object 
 7   product_id                38279 non-null  object 
 8   seller_id                 38279 non-null  object 
 9   price                     38279 non-null  float64
 10  shipping_charges          38279 non-null  float64
 11  payment_sequential        38279 non-null  int64  
 12  payment_type              38279 non-null  object 
 13  payment_installments      38279 non-null  int64  
 14  paymen

In [87]:
## Convert time data with the datetime format
tr_data[["order_purchase_timestamp", "order_approved_at", 
         "order_delivered_timestamp", "order_estimated_delivery_date"]] = tr_data[["order_purchase_timestamp", "order_approved_at", 
                                                                                   "order_delivered_timestamp", "order_estimated_delivery_date"]].apply(pd.to_datetime)

ts_data[["order_purchase_timestamp", "order_approved_at"]] = ts_data[["order_purchase_timestamp", "order_approved_at"]].apply(pd.to_datetime)

In [88]:
print('Training Data Info\n')
print(tr_data.info())
print('Null Info in Training Data')
print(tr_data.isna().sum())
print('\nTesting Data')
print(ts_data.info())
print('Null Info in Testing Data')
print(ts_data.isna().sum())

Training Data Info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       89316 non-null  object        
 1   customer_id                    89316 non-null  object        
 2   order_status                   89316 non-null  object        
 3   order_purchase_timestamp       89316 non-null  datetime64[ns]
 4   order_approved_at              89307 non-null  datetime64[ns]
 5   order_delivered_timestamp      87427 non-null  datetime64[ns]
 6   order_estimated_delivery_date  89316 non-null  datetime64[ns]
 7   customer_zip_code_prefix       89316 non-null  int64         
 8   customer_city                  89316 non-null  object        
 9   customer_state                 89316 non-null  object        
 10  product_id                     89316 non-null  object        


In [89]:
tr_data = tr_data[tr_data.order_status == 'delivered']
tr_data

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,...,shipping_charges,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,58125,varzea paulista,SP,...,84.65,1,credit_card,1,259.14,toys,491.0,19.0,12.0,16.0
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24,3112,armacao dos buzios,RJ,...,23.79,1,credit_card,8,382.39,watches_gifts,440.0,18.0,14.0,17.0
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08,4119,jandira,SP,...,17.38,1,credit_card,4,249.25,costruction_tools_garden,2200.0,16.0,16.0,16.0
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19,18212,uberlandia,MG,...,30.72,1,credit_card,2,27.79,toys,1450.0,68.0,3.0,48.0
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18,88868,ilhabela,SP,...,30.66,1,credit_card,1,76.15,toys,300.0,17.0,4.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89311,zlcLnXGgAabl,bfJtzkBGksdY,delivered,2018-08-19 17:25:36,2018-08-21 04:35:30,2018-08-30 14:47:46,2018-09-04,94060,amparo,SP,...,188.65,1,wallet,1,128.00,toys,700.0,21.0,14.0,14.0
89312,IlTAoIkILwrq,4hjsCpj6FC7g,delivered,2017-12-15 11:20:27,2017-12-16 02:37:32,2017-12-20 19:23:40,2018-01-04,99835,itapevi,SP,...,1.64,1,wallet,1,106.86,toys,600.0,16.0,16.0,16.0
89313,uA6oXfftGVmT,DLn56oKHl9bv,delivered,2018-07-24 06:39:14,2018-07-24 11:20:39,2018-07-27 01:41:40,2018-08-01,2540,sao paulo,SP,...,4.09,1,credit_card,1,376.39,health_beauty,250.0,22.0,10.0,18.0
89314,v3QtU6xlGDJp,auSfRYUQUOIq,delivered,2018-08-11 08:04:22,2018-08-11 08:24:40,2018-08-14 21:43:51,2018-08-17,12288,paulinia,SP,...,113.01,1,voucher,1,48.05,toys,250.0,16.0,2.0,11.0


In [90]:
tr_prods = tr_products.drop_duplicates() #-->drop duplicates on product
tr_data = tr_orders.merge(tr_customers, on="customer_id", how="left")
tr_data = tr_data.merge(tr_items, on="order_id", how="left")
tr_data = tr_data.merge(tr_payments, on="order_id", how="left")
tr_data = tr_data.merge(tr_prods, on="product_id", how="left")
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       89316 non-null  object 
 1   customer_id                    89316 non-null  object 
 2   order_status                   89316 non-null  object 
 3   order_purchase_timestamp       89316 non-null  object 
 4   order_approved_at              89307 non-null  object 
 5   order_delivered_timestamp      87427 non-null  object 
 6   order_estimated_delivery_date  89316 non-null  object 
 7   customer_zip_code_prefix       89316 non-null  int64  
 8   customer_city                  89316 non-null  object 
 9   customer_state                 89316 non-null  object 
 10  product_id                     89316 non-null  object 
 11  seller_id                      89316 non-null  object 
 12  price                          89316 non-null 

In [91]:
## Convert time data with the datetime format
tr_data[["order_purchase_timestamp", "order_approved_at", 
         "order_delivered_timestamp", "order_estimated_delivery_date"]] = tr_data[["order_purchase_timestamp", "order_approved_at", 
                                                                                   "order_delivered_timestamp", "order_estimated_delivery_date"]].apply(pd.to_datetime)

ts_data[["order_purchase_timestamp", "order_approved_at"]] = ts_data[["order_purchase_timestamp", "order_approved_at"]].apply(pd.to_datetime)

In [92]:
# Let's assume that null in approved_at in the testing data will result in delays
ts_data['order_approved_at'] = ts_data['order_approved_at'].fillna(ts_data['order_purchase_timestamp'] + timedelta(days=25))

In [93]:
tr_data = tr_data.drop(['order_status'], axis=1)
tr_data.head()

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,product_id,...,shipping_charges,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,58125,varzea paulista,SP,90K0C1fIyQUf,...,84.65,1,credit_card,1,259.14,toys,491.0,19.0,12.0,16.0
1,v6px92oS8cLG,PxA7fv9spyhx,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24,3112,armacao dos buzios,RJ,qejhpMGGVcsl,...,23.79,1,credit_card,8,382.39,watches_gifts,440.0,18.0,14.0,17.0
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08,4119,jandira,SP,qUS5d2pEAyxJ,...,17.38,1,credit_card,4,249.25,costruction_tools_garden,2200.0,16.0,16.0,16.0
3,bwJVWupf2keN,EOEsCQ6QlpIg,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19,18212,uberlandia,MG,639iGvMyv0De,...,30.72,1,credit_card,2,27.79,toys,1450.0,68.0,3.0,48.0
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18,88868,ilhabela,SP,1lycYGcsic2F,...,30.66,1,credit_card,1,76.15,toys,300.0,17.0,4.0,12.0


In [94]:
tr_data.isna().sum()

order_id                            0
customer_id                         0
order_purchase_timestamp            0
order_approved_at                   9
order_delivered_timestamp        1889
order_estimated_delivery_date       0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
product_id                          0
seller_id                           0
price                               0
shipping_charges                    0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
product_category_name             308
product_weight_g                   15
product_length_cm                  15
product_height_cm                  15
product_width_cm                   15
dtype: int64

In [95]:
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       89316 non-null  object        
 1   customer_id                    89316 non-null  object        
 2   order_purchase_timestamp       89316 non-null  datetime64[ns]
 3   order_approved_at              89307 non-null  datetime64[ns]
 4   order_delivered_timestamp      87427 non-null  datetime64[ns]
 5   order_estimated_delivery_date  89316 non-null  datetime64[ns]
 6   customer_zip_code_prefix       89316 non-null  int64         
 7   customer_city                  89316 non-null  object        
 8   customer_state                 89316 non-null  object        
 9   product_id                     89316 non-null  object        
 10  seller_id                      89316 non-null  object        
 11  price          

In [96]:
## Labeling and Feature Engineering
tr_data.select_dtypes(include=["datetime64"]).columns

Index(['order_purchase_timestamp', 'order_approved_at',
       'order_delivered_timestamp', 'order_estimated_delivery_date'],
      dtype='object')

In [97]:
tr_data = tr_data.dropna(subset=['order_approved_at', 'order_delivered_timestamp'])

In [98]:
tr_data.isna().sum()

order_id                           0
customer_id                        0
order_purchase_timestamp           0
order_approved_at                  0
order_delivered_timestamp          0
order_estimated_delivery_date      0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
product_id                         0
seller_id                          0
price                              0
shipping_charges                   0
payment_sequential                 0
payment_type                       0
payment_installments               0
payment_value                      0
product_category_name            289
product_weight_g                  15
product_length_cm                 15
product_height_cm                 15
product_width_cm                  15
dtype: int64

In [99]:
tr_data['is_late_s'] = (tr_data['order_delivered_timestamp'] > tr_data['order_estimated_delivery_date']).astype(int)
tr_data.head()

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state,product_id,...,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,is_late_s
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,58125,varzea paulista,SP,90K0C1fIyQUf,...,1,credit_card,1,259.14,toys,491.0,19.0,12.0,16.0,0
1,v6px92oS8cLG,PxA7fv9spyhx,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24,3112,armacao dos buzios,RJ,qejhpMGGVcsl,...,1,credit_card,8,382.39,watches_gifts,440.0,18.0,14.0,17.0,0
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08,4119,jandira,SP,qUS5d2pEAyxJ,...,1,credit_card,4,249.25,costruction_tools_garden,2200.0,16.0,16.0,16.0,0
3,bwJVWupf2keN,EOEsCQ6QlpIg,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19,18212,uberlandia,MG,639iGvMyv0De,...,1,credit_card,2,27.79,toys,1450.0,68.0,3.0,48.0,0
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18,88868,ilhabela,SP,1lycYGcsic2F,...,1,credit_card,1,76.15,toys,300.0,17.0,4.0,12.0,0


In [102]:
tr_data["is_late_s"].value_counts()

is_late_s
0    80680
1     6738
Name: count, dtype: int64

In [103]:
import skimpy

skimpy.skim(tr_data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 87418  │ │ string      │ 8     │                                                          │
│ │ Number of columns │ 23     │ │ float64     │ 7     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 4     │                                                          │
│                                │ int32       │ 4     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name            ┃ NA  ┃ NA %  ┃ mean     ┃ sd      ┃ p0    ┃ p25   ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ customer_zip_code_pref │   0 │     0 │    40480 │   31200 │  1003 │  9783 │ 35450 │ 69930 │ 99990 │ ▇▃▃▃▃▃ │  │
│ │ ix                     │     │       │          │         │       │       │       │       │       │        │  │
│ │ price                  │   0 │     0 │    342.3 │   559.8 │  0.85 │ 59.65 │ 136.9 │ 399.2 │  6735 │   ▇    │  │
│ │ shipping_charges       │   0 │     0 │    44.24 │   37.66 │     0 │ 20.04 │ 35.01 │ 57.14 │ 409.7 │   ▇▁   │  │
│ │ payment_sequential     │   0 │     0 │    1.097 │  0.7422 │     1 │     1 │     1 │     1 │    26 │   ▇    │  │
│ │ payment_installments   │   0 │     0 │    2.964 │   2.795 │     0 │     1 │     2 │     4 │    24 │  ▇▂▁   │  │
│ │ payment_value          │   0 │     0 │    268.8 │   344.8 │     0 │ 84.32 │ 171.9 │ 313.6 │  7275 │   ▇    │  │
│ │ product_weight_g       │  15 │  0.02 │     2082 │    3735 │     0 │   300 │   700 │  1800 │ 40420 │   ▇▁   │  │
│ │ product_length_cm      │  15 │  0.02 │     30.2 │   16.05 │     7 │    18 │    25 │    38 │   105 │  ▇▅▃▁  │  │
│ │ product_height_cm      │  15 │  0.02 │    16.53 │   13.37 │     2 │     8 │    13 │    20 │   105 │   ▇▃   │  │
│ │ product_width_cm       │  15 │  0.02 │    23.03 │   11.67 │     6 │    15 │    20 │    30 │   118 │  ▇▃▁   │  │
│ │ is_late_s              │   0 │     0 │  0.07708 │  0.2667 │     0 │     0 │     0 │     0 │     1 │ ▇    ▁ │  │
│ └────────────────────────┴─────┴───────┴──────────┴─────────┴───────┴───────┴───────┴───────┴───────┴────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA  ┃ NA %   ┃ first                   ┃ last                    ┃ frequency   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩  │
│ │ order_purchase_timestamp  │   0 │      0 │   2016-10-03 09:44:50   │   2018-08-29 14:52:00   │ None        │  │
│ │ order_approved_at         │   0 │      0 │   2016-10-04 09:43:32   │   2018-08-29 15:05:22   │ None        │  │
│ │ order_delivered_timestamp │   0 │      0 │   2016-10-11 13:46:32   │   2018-10-17 13:22:46   │ None        │  │
│ │ order_estimated_delivery_ │   0 │      0 │       201